In [1]:
# install PyDrive
!pip install -U -q PyDrive

# imports
from __future__ import print_function
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from tensorflow.contrib import lite
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Flatten, Add, Concatenate, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D
from keras import backend as K
from scipy.signal import stft
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Using TensorFlow backend.


In [0]:
# links for data loading
link_img = "https://drive.google.com/open?id=1lAxQhHf8oNjTZ84_empUhcaZOL2lWqwu"
link_emg = "https://drive.google.com/open?id=181QG5teO-btOlGMBkD6negYWBz2M6XKr"
link_lbl = "https://drive.google.com/open?id=1k5UIqXI_pdu99j1zk1C2Rn7jnsGdgQs_"
link_frm = "https://drive.google.com/open?id=1iQyZL8z0A12iDQqOR-O2PvUcrf9h9Q31"
link_evt = "https://drive.google.com/open?id=1YWR4HqWiK3EU7ZVCTsXefVKpH7R8d8QC"
link_100 = "https://drive.google.com/open?id=1lTS3kSo_M2Zj3NoRB_pxS7GRgqj_i0fu"
link_150 = "https://drive.google.com/open?id=1RWBe7MOEkDM8Jjcs5NaGbXe_LdoHar43"
link_200 = "https://drive.google.com/open?id=1sfaVY7VXXUD3x0ak8iyUIko217Dwrivd"
link_250 = "https://drive.google.com/open?id=1R9481MZIqPxr40wLoCB4nJAtTKZRiBni"

# names for data loading
name_img = "cc19_hand_gestures_10p_img.npy"
name_emg = "cc19_hand_gestures_10p_emg.npy"
name_lbl = "cc19_hand_gestures_10p_lbl.npy"
name_frm = "full_dataset_frames.pkl"
name_evt = "full_dataset_events.pkl"
name_100 = "fusion_features_100ms.pkl"
name_150 = "fusion_features_150ms.pkl"
name_200 = "fusion_features_200ms.pkl"
name_250 = "fusion_features_250ms.pkl"

# names for h5 data saving
name_emg = "cnn_emg.h5"
name_dvs_evs = "cnn_dvs_evs.h5"
name_dav_evs = "cnn_dav_evs.h5"
name_dav_aps = "cnn_dav_aps.h5"
name_emg_dvs_evs = "cnn_emg+dvs_evs.h5"
name_emg_dav_evs = "cnn_emg+dav_evs.h5"
name_emg_dav_aps = "cnn_emg+dav_aps.h5"

# names for tflite data saving
tflite_emg = "cnn_emg.tflite"
tflite_dvs_evs = "cnn_dvs_evs.tflite"
tflite_dav_evs = "cnn_dav_evs.tflite"
tflite_dav_aps = "cnn_dav_aps.tflite"
tflite_emg_dvs_evs = "cnn_emg+dvs_evs.tflite"
tflite_emg_dav_evs = "cnn_emg+dav_evs.tflite"
tflite_emg_dav_aps = "cnn_emg+dav_aps.tflite"

In [0]:
# hyper-parameters definition
data_base = 200
k_splits = 5
batch_size = 128
num_classes = 5
div_epochs = 50
emg_epochs = 50
fus_epochs = 50
seed = 23
np.random.seed(seed)

In [4]:
# fuction to load data
def load_data(link, name):
    fluff, id = link.split('=')
    print("--- loading data: " + name + " from id -> " + id + " ---")
    downloaded = drive.CreateFile({"id":id}) 
    downloaded.GetContentFile(name)  
    data = np.load(name, allow_pickle=True)
    
    labels = data["lbl"]
    emg = data["emg"]
    dvs_evs = data["dvs_evs_frames"]
    dav_evs = data["dav_evs_frames"]
    dav_aps = data["dav_aps_frames"]
    
    print("labels.shape = ", labels.shape)
    print("emg.shape = ", emg.shape)
    print("dvs_evs.shape = ", dvs_evs.shape)
    print("dav_evs.shape = ", dav_evs.shape)
    print("dav_aps.shape = ", dav_aps.shape)
    print("\n")
    
    return labels, emg, dvs_evs, dav_evs, dav_aps

# load data
if data_base == 100:
    data_labels, data_emg, data_dvs_evs, data_dav_evs, data_dav_aps = load_data(link_100, name_100)
elif data_base == 150:
    data_labels, data_emg, data_dvs_evs, data_dav_evs, data_dav_aps = load_data(link_150, name_150)
elif data_base == 200:
    data_labels, data_emg, data_dvs_evs, data_dav_evs, data_dav_aps = load_data(link_200, name_200)
elif data_base == 250:
    data_labels, data_emg, data_dvs_evs, data_dav_evs, data_dav_aps = load_data(link_250, name_250)

--- loading data: fusion_features_200ms.pkl from id -> 1sfaVY7VXXUD3x0ak8iyUIko217Dwrivd ---
labels.shape =  (6750, 1)
emg.shape =  (6750, 16)
dvs_evs.shape =  (6750, 60, 60)
dav_evs.shape =  (6750, 60, 60)
dav_aps.shape =  (6750, 60, 60)




In [5]:
# function to normalize and standardize data
def norm_stand(data):
    print("--- normalize and standardize data ---")
    
    print("data_max = ", np.max(data))
    print("data_min = ", np.min(data))
    print("data_mean = ", np.mean(data))
    print("data_std = ", np.std(data))
    
    data = data.astype(float)

    # normalize
    data_max = np.max(data)
    data_min = np.min(data)
    for i in range(len(data)):
        data[i] = (data[i] - data_min) / (data_max - data_min)
        
    # standardize
    data_mean = np.mean(data)
    data_std = np.std(data) + 1e-15
    data -= data_mean
    data /= data_std
    
    print("---")
    print("new_data_max = ", np.max(data))
    print("new_data_min = ", np.min(data))
    print("new_data_mean = ", np.mean(data))
    print("new_data_std = ", np.std(data))
    print("\n")
    
    return data
    
# normalize and standardize data
data_emg = norm_stand(data_emg)
data_dvs_evs = norm_stand(data_dvs_evs)
data_dav_evs = norm_stand(data_dav_evs)
data_dav_aps = norm_stand(data_dav_aps)

--- normalize and standardize data ---
data_max =  68.71754309696473
data_min =  0.0
data_mean =  9.989849023500414
data_std =  8.00750450155642
---
new_data_max =  7.334081930525158
new_data_min =  -1.2475608376565506
new_data_mean =  -1.1579214953127558e-17
new_data_std =  0.9999999999999913


--- normalize and standardize data ---
data_max =  255
data_min =  0
data_mean =  5.02497621399177
data_std =  18.176710732725073
---
new_data_max =  13.752489515936109
new_data_min =  -0.27645134963526835
new_data_mean =  -6.117059488114522e-16
new_data_std =  0.9999999999999869


--- normalize and standardize data ---
data_max =  240
data_min =  0
data_mean =  12.66215390946502
data_std =  23.17336329401839
---
new_data_max =  9.810308637814954
new_data_min =  -0.5464098477553875
new_data_mean =  6.146270689473549e-16
new_data_std =  0.9999999999999903


--- normalize and standardize data ---
data_max =  250
data_min =  2
data_mean =  184.0460073251029
data_std =  46.54963776709588
---
new_da

In [0]:
# data format convertion
data_labels = keras.utils.to_categorical(data_labels, num_classes)
data_emg = data_emg.reshape(data_emg.shape[0], data_emg.shape[1], 1)
data_dvs_evs = data_dvs_evs.reshape(data_dvs_evs.shape[0], data_dvs_evs.shape[1], data_dvs_evs.shape[2], 1)
data_dav_evs = data_dav_evs.reshape(data_dav_evs.shape[0], data_dav_evs.shape[1], data_dav_evs.shape[2], 1)
data_dav_aps = data_dav_aps.reshape(data_dvs_evs.shape[0], data_dvs_evs.shape[1], data_dvs_evs.shape[2], 1)

vid_input_shape = (data_dvs_evs.shape[1], data_dvs_evs.shape[2], 1)
emg_input_shape = (data_emg.shape[1], 1)

In [0]:
# function to shuffle data
def unison_shuffled_copies(a, b, c, d, e):
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p], d[p], e[p]

# shuffle data
data_labels, data_emg, data_dvs_evs, data_dav_evs, data_dav_aps = unison_shuffled_copies(data_labels, data_emg, data_dvs_evs, data_dav_evs, data_dav_aps)

In [0]:
# cnn model for vid
def create_vid_lenet():
    print("--- vid_cnn: lenet ---")
    model = keras.Sequential()
    model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=vid_input_shape))
    model.add(AveragePooling2D())
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D())
    model.add(Conv2D(filters=120, kernel_size=(5, 5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=84, activation='relu'))
    model.add(Dense(units=num_classes, activation = 'softmax'))
    
    #model.summary()
    
    return model
  
def create_vid_cc19():
    print("--- vid_cnn: cc19 ---")
    model_img = Sequential()
    model_img.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu', input_shape=vid_input_shape))
    model_img.add(Conv2D(filters=8, kernel_size=(3, 3), activation='relu'))
    model_img.add(MaxPooling2D(pool_size=(2, 2)))
    model_img.add(Dropout(0.25))
    model_img.add(Flatten())
    model_img.add(Dense(50, activation='relu'))
    model_img.add(Dropout(0.5))
    model_img.add(Dense(num_classes, activation='softmax'))

    #model_img.summary()
    
    return model_img

In [0]:
# cnn model for emg
def create_emg_lenet():
    print("--- emg_cnn: lenet ---")
    model = keras.Sequential()
    model.add(Conv1D(filters=6, kernel_size=3, activation='relu', input_shape=emg_input_shape))
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=120, activation='relu'))
    model.add(Dense(units=84, activation='relu'))
    model.add(Dense(units=num_classes, activation = 'softmax'))
    
    #model.summary()
    
    return model

def create_emg_cc19():
    print("--- emg_cnn: cc19 ---")
    model_emg = Sequential()
    model_emg.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=emg_input_shape))
    model_emg.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model_emg.add(Dropout(0.5))
    model_emg.add(MaxPooling1D(pool_size=2))
    model_emg.add(Flatten())
    model_emg.add(Dense(100, activation='relu'))
    model_emg.add(Dense(num_classes, activation='softmax'))

    #model_emg.summary()
    
    return model_emg

In [10]:
### data_dvs_evs ###

# evaluate a single model
def evaluate_dvs_evs(trainX, trainy, testX, testy):
    # compile the model
    model_dvs_evs.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_dvs_evs.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_dvs_evs.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_dvs_evs, test_acc

# prepare the k-fold cross-validation configuration
n_folds = 5
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_dvs_evs):
    # reset variables
    %reset_selective -f model_div
    
    # select samples
    trainX, trainy = data_dvs_evs[train_ix], data_labels[train_ix]
    testX, testy = data_dvs_evs[test_ix], data_labels[test_ix]
    
    # create model
    model_dvs_evs = create_vid_lenet()
    
    # evaluate model
    model, test_acc = evaluate_dvs_evs(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- vid_cnn: lenet ---
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
1350/1350 [==============================] - 0s 72us/step
>0.893
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 78us/step
>0.901
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 67us/step
>0.893
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 76us/step
>0.902
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 68us/step
>0.902
Estimated Accuracy 0.899 (0.004)


In [11]:
### data_dav_evs ###

# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy):
    # compile the model
    model_dav_evs.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_dav_evs.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_dav_evs.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_dav_evs, test_acc

# prepare the k-fold cross-validation configuration
n_folds = 5
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_dav_evs):
    # reset variables
    %reset_selective -f model_div
    
    # select samples
    trainX, trainy = data_dav_evs[train_ix], data_labels[train_ix]
    testX, testy = data_dav_evs[test_ix], data_labels[test_ix]
    
    # create model
    model_dav_evs = create_vid_lenet()
    
    # evaluate model
    model, test_acc = evaluate_model(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 81us/step
>0.901
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 71us/step
>0.909
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 72us/step
>0.931
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 67us/step
>0.909
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 68us/step
>0.916
Estimated Accuracy 0.913 (0.010)


In [12]:
### data_dav_aps ###

# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy):
    # compile the model
    model_dav_aps.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_dav_aps.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_dav_aps.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_dav_aps, test_acc

# prepare the k-fold cross-validation configuration
n_folds = 5
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_dav_aps):
    # reset variables
    %reset_selective -f model_div
    
    # select samples
    trainX, trainy = data_dav_aps[train_ix], data_labels[train_ix]
    testX, testy = data_dav_aps[test_ix], data_labels[test_ix]
    
    # create model
    model_dav_aps = create_vid_lenet()
    
    # evaluate model
    model, test_acc = evaluate_model(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 82us/step
>0.918
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 68us/step
>0.930
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 78us/step
>0.919
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 81us/step
>0.926
--- vid_cnn: lenet ---
1350/1350 [==============================] - 0s 72us/step
>0.919
Estimated Accuracy 0.922 (0.005)


In [13]:
### data_emg ###

# evaluate a single model
def evaluate_emg(trainX, trainy, testX, testy):
    # compile the model
    model_emg.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_emg.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_emg.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_emg, test_acc

# prepare the k-fold cross-validation configuration
n_folds = 5
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_emg):
    # reset variables
    %reset_selective -f model_emg
    
    # select samples
    trainX, trainy = data_emg[train_ix], data_labels[train_ix]
    testX, testy = data_emg[test_ix], data_labels[test_ix]
    
    # create model
    model_emg = create_emg_lenet()
    
    # evaluate model
    model, test_acc = evaluate_emg(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- emg_cnn: lenet ---
1350/1350 [==============================] - 0s 55us/step
>0.814
--- emg_cnn: lenet ---
1350/1350 [==============================] - 0s 54us/step
>0.832
--- emg_cnn: lenet ---
1350/1350 [==============================] - 0s 52us/step
>0.832
--- emg_cnn: lenet ---
1350/1350 [==============================] - 0s 52us/step
>0.831
--- emg_cnn: lenet ---
1350/1350 [==============================] - 0s 52us/step
>0.817
Estimated Accuracy 0.825 (0.008)


In [0]:
# mlp model for fus
def create_mlp_fus(model_vid, model_emg):
    print("--- fus_mlp ---")
    mergedOut = Concatenate()([model_vid.output, model_emg.output])
    mergedOut = Dense(5, activation='softmax')(mergedOut)
    model_fus = Model([model_vid.input, model_emg.input], mergedOut)
    
    # freeze the layers except the last dense
    for layer in model_fus.layers[:len(model_fus.layers)-1]:
        layer.trainable = False

    # check the status of the layers
    """
    for layer in model_fus.layers:
        print(layer, layer.trainable)
    model_fus.summary()
    """
    
    return model_fus

In [15]:
### data_emg + data_dvs_evs ###

# evaluate a single model
def evaluate_emg_dvs_evs(trainX, trainy, testX, testy):
    # compile the model
    model_emg_dvs_evs.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_emg_dvs_evs.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_emg_dvs_evs.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_emg_dvs_evs, test_acc

# prepare the k-fold cross-validation configuration
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_emg):
    # reset variables
    %reset_selective -f model_emg_dvs_evs
    
    # select samples
    trainX, trainy = [data_dvs_evs[train_ix], data_emg[train_ix]], data_labels[train_ix]
    testX, testy = [data_dvs_evs[test_ix], data_emg[test_ix]], data_labels[test_ix]
    
    # create model
    model_emg_dvs_evs = create_mlp_fus(model_dvs_evs, model_emg)
    
    # evaluate model
    model, test_acc = evaluate_emg_dvs_evs(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- fus_mlp ---
1350/1350 [==============================] - 0s 94us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 93us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 107us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 95us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 94us/step
>0.913
Estimated Accuracy 0.983 (0.035)


In [16]:
### data_emg + data_dav_evs ###

# evaluate a single model
def evaluate_emg_dav_evs(trainX, trainy, testX, testy):
    # compile the model
    model_emg_dav_evs.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_emg_dav_evs.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_emg_dav_evs.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_emg_dav_evs, test_acc

# prepare the k-fold cross-validation configuration
n_folds = 5
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_emg):
    # reset variables
    %reset_selective -f model_emg_dav_evs
    
    # select samples
    trainX, trainy = [data_dav_evs[train_ix], data_emg[train_ix]], data_labels[train_ix]
    testX, testy = [data_dav_evs[test_ix], data_emg[test_ix]], data_labels[test_ix]
    
    # create model
    model_emg_dav_evs = create_mlp_fus(model_dav_evs, model_emg)
    
    # evaluate model
    model, test_acc = evaluate_emg_dav_evs(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- fus_mlp ---
1350/1350 [==============================] - 0s 95us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 96us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 94us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 95us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 94us/step
>0.931
Estimated Accuracy 0.986 (0.028)


In [17]:
### data_emg + data_dav_aps ###

# evaluate a single model
def evaluate_emg_dav_aps(trainX, trainy, testX, testy):
    # compile the model
    model_emg_dav_aps.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
    
    # fit the model
    model_emg_dav_aps.fit(trainX, trainy,
            batch_size=batch_size,
            epochs=div_epochs,
            verbose=0,
            validation_data=(testX, testy))
    
    # evaluate the model
    scores, test_acc = model_emg_dav_aps.evaluate(testX, testy, verbose=1)
    #print("%s: %.2f%%" % (model_vid.metrics_names[1], scores[1]*100))
    
    return model_emg_dav_aps, test_acc

# prepare the k-fold cross-validation configuration
n_folds = 5
kfold = KFold(n_splits = k_splits, shuffle=False, random_state=None)

# cross validation estimation of performance
scores, members = list(), list()
for train_ix, test_ix in kfold.split(data_emg):
    # reset variables
    %reset_selective -f model_emg_dav_aps
    
    # select samples
    trainX, trainy = [data_dav_aps[train_ix], data_emg[train_ix]], data_labels[train_ix]
    testX, testy = [data_dav_aps[test_ix], data_emg[test_ix]], data_labels[test_ix]
    
    # create model
    model_emg_dav_aps = create_mlp_fus(model_dav_aps, model_emg)
    
    # evaluate model
    model, test_acc = evaluate_emg_dav_aps(trainX, trainy, testX, testy)
    print('>%.3f' % test_acc)
    scores.append(test_acc)
    members.append(model)

# summarize expected performance
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

--- fus_mlp ---
1350/1350 [==============================] - 0s 92us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 96us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 98us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 98us/step
>1.000
--- fus_mlp ---
1350/1350 [==============================] - 0s 97us/step
>0.936
Estimated Accuracy 0.987 (0.025)


In [0]:
"""
# function to save keras model on google drive
def save_model(model, model_name):
    # create on colab directory
    model.save(model_name)    
    model_file = drive.CreateFile({'title': model_name})
    model_file.SetContentFile(model_name)
    model_file.Upload()

    # download to google drive
    drive.CreateFile({'id': model_file.get('id')})
    
# save keras models on google drive
save_model(model_emg, name_emg)
save_model(model_dvs_evs, name_dvs_evs)
save_model(model_dav_evs, name_dav_evs)
save_model(model_dav_aps, name_dav_aps)
save_model(model_emg_dvs_evs, name_emg_dvs_evs)
save_model(model_emg_dav_evs, name_emg_dav_evs)
save_model(model_emg_dav_aps, name_emg_dav_aps)
"""

In [0]:
"""
# links for models loading
link_emg = "https://drive.google.com/open?id=1-47ifKsKx5PFg4IgcsjGmEoOFwcG5or-"
link_dvs_evs = "https://drive.google.com/open?id=1OrUHw8d0XGrcjlYSCrXpSigCEm0Ck30v"
link_dav_evs = "https://drive.google.com/open?id=1XeQUUTnPcJxJDVy8N17HG93oDpoprn7q"
link_dav_aps = "https://drive.google.com/open?id=1SQpQ2_ncZsKdX6RDcGQnrfvKdzIFnfSW"
link_emg_dvs_evs = "https://drive.google.com/open?id=1pU8kev4Aj5D6YapvRErqPmMEBVlPdlOi"
link_emg_dav_evs = "https://drive.google.com/open?id=1NxQi-51-WgvVsnrGp1jur8mL9yghzzJd"
link_emg_dav_aps = "https://drive.google.com/open?id=1lXCSsP_vRqi4d5tisoSNVXE58Lgc5kUA"

# names for models loading
name_emg = "cnn_200_emg.h5"
name_dvs_evs = "cnn_200_dvs_evs.h5"
name_dav_evs = "cnn_200_dav_evs.h5"
name_dav_aps = "cnn_200_dav_aps.h5"
name_emg_dvs_evs = "cnn_200_emg+dvs_evs.h5"
name_emg_dav_evs = "cnn_200_emg+dav_evs.h5"
name_emg_dav_aps = "cnn_200_emg+dav_aps.h5"

# fuction to load h5 models from google drive
def load_h5_model(link, name):
    fluff, id = link.split('=')
    print("--- loading model: " + name + " from id -> " + id + " ---")
    downloaded = drive.CreateFile({"id":id})
    downloaded.GetContentFile(name)  
    model = load_model(name)

    #model.summary()
  
# load models
load_h5_model(link_emg, name_emg)
load_h5_model(link_dvs_evs, name_dvs_evs)
load_h5_model(link_dav_evs, name_dav_evs)
load_h5_model(link_dav_aps, name_dav_aps)
load_h5_model(link_emg_dvs_evs, name_emg_dvs_evs)
load_h5_model(link_emg_dav_evs, name_emg_dav_evs)
load_h5_model(link_emg_dav_aps, name_emg_dav_aps)

# function to convert h5 model to tflite model
def save_tflite(name, file_name):
    print("--- saving model: " + name + " into -> " + file_name + " ---")
    converter = lite.TFLiteConverter.from_keras_model_file(name)
    model_tflite = converter.convert()
    file = open(file_name, 'wb')
    file.write(model_tflite)

    # save tflite model on google drive
    # create on colab directory
    model_file = drive.CreateFile({'title': file_name})
    model_file.SetContentFile(file_name)
    model_file.Upload()

    # download to google drive
    drive.CreateFile({'id': model_file.get('id')})
    
# convert h5 model to tflite model
save_tflite(name_emg, tflite_emg)
save_tflite(name_dvs_evs, tflite_dvs_evs)
save_tflite(name_dav_evs, tflite_dav_evs)
save_tflite(name_dav_aps, tflite_dav_aps)
save_tflite(name_emg_dvs_evs, tflite_emg_dvs_evs)
save_tflite(name_emg_dav_evs, tflite_emg_dav_evs)
save_tflite(name_emg_dav_aps, tflite_emg_dav_aps)
"""